# 0. Introduction

In [0]:
%matplotlib inline

Example based on: Deep Learning with Python by Francois Chollet:
https://www.manning.com/books/deep-learning-with-python

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np

# 1. Load data

The MNIST database of handwritten digit has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.
It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting.

In [0]:
!rm -rf myModel

In [8]:
#from tensorflow.models import mnist
import tensorflow as tf
import numpy as np

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W, name=None):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME', name=name)


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')


def conv_layer(input, shape, name=None):
    W = weight_variable(shape)
    b = bias_variable([shape[3]])
    return tf.nn.relu(conv2d(input, W, name=name) + b)


def full_layer(input, size, name=None):
    in_size = int(input.get_shape()[1])
    W = weight_variable([in_size, size])
    b = bias_variable([size])
    return tf.matmul(input, W, name=name) + b

DATA_DIR = '/tmp/data'
MINIBATCH_SIZE = 50
STEPS = 5000


mnist = input_data.read_data_sets(DATA_DIR, one_hot=True)

x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

x_image = tf.reshape(x, [-1, 28, 28, 1])
conv1 = conv_layer(x_image, shape=[5, 5, 1, 32], name="inputLayer")
conv1_pool = max_pool_2x2(conv1)

conv2 = conv_layer(conv1_pool, shape=[5, 5, 32, 64])
conv2_pool = max_pool_2x2(conv2)

conv2_flat = tf.reshape(conv2_pool, [-1, 7*7*64])
full_1 = tf.nn.relu(full_layer(conv2_flat, 1024))

keep_prob = tf.placeholder(tf.float32)
full1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob)

y_conv = full_layer(full1_drop, 10, name="inferenceLayer")

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(STEPS):
        batch = mnist.train.next_batch(MINIBATCH_SIZE)

        if i % 100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x: batch[0], y_: batch[1],
                                                           keep_prob: 1.0})
            print("step {}, training accuracy {}".format(i, train_accuracy))

        sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    X = mnist.test.images.reshape(10, 1000, 784)
    Y = mnist.test.labels.reshape(10, 1000, 10)
    test_accuracy = np.mean(
        [sess.run(accuracy, feed_dict={x: X[i], y_: Y[i], keep_prob: 1.0}) for i in range(10)])
    builder = tf.saved_model.builder.SavedModelBuilder("myModel")
    builder.add_meta_graph_and_variables(sess,
                                  ["myTag"]
                                  
                                  )
    builder.save()

print("test accuracy: {}".format(test_accuracy))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.07999999821186066
step 100, training accuracy 0.8199999928474426
step 200, training accuracy 0.9599999785423279
step 300, training accuracy 0.9399999976158142
step 400, training accuracy 0.9599999785423279
step 500, training accuracy 0.9399999976158142
step 600, training accuracy 0.9399999976158142
step 700, training accuracy 0.9800000190734863
step 800, training accuracy 0.9200000166893005
step 900, training accuracy 0.9800000190734863
step 1000, training accuracy 0.9800000190734863
step 1100, training accuracy 0.9800000190734863
step 1200, training accuracy 0.9800000190734863
step 1300, training accuracy 1.0
step 1400, training accuracy 0.9800000190734863
step 1500, training accuracy 1.0
step 1600, training accuracy 0.9800000190734863
step 1700, training accuracy 1.0
st

In [9]:
! ls -l myModel

total 220
-rw-r--r-- 1 root root 220698 Jan 23 09:56 saved_model.pb
drwxr-xr-x 2 root root   4096 Jan 23 09:56 variables


In [10]:
!ls -l myModel/variables

total 115128
-rw-r--r-- 1 root root 117886848 Jan 23 09:56 variables.data-00000-of-00001
-rw-r--r-- 1 root root      2559 Jan 23 09:56 variables.index


# 2. Inspect data

Try to get a feel for the data you are using to train and test your neural network. 

## Training data

- Training data will be used to train our neural network to recognize hand-written digits.
- MNIST provides 60000 labeled training images, each 28x28 pixels

In [0]:
train_images.shape

In [0]:
train_labels.shape

In [0]:
def show_image(images, labels, index):
    img = images[index].reshape((28,28))
    label = labels[index]
    plt.imshow(img)
    plt.title(label)
    plt.show()

In [0]:
show_image(train_images, train_labels, 10)

## Test data

- Test data will be used to validate how good our network performs on data it has never seen.
- MNIST provides 10000 test images, each 28x28.
- It's important to note that these should never be used in the training cycle. A 'test set' should never contain images the network has already seen during training. (read more: [Model Selection and Train/Validation/Test Sets](https://www.coursera.org/lecture/machine-learning/model-selection-and-train-validation-test-sets-QGKbr) and [How (and why) to create a good validation set](https://www.fast.ai/2017/11/13/validation-sets/))

In [0]:
test_images.shape

In [0]:
test_labels.shape

In [0]:
show_image(test_images, test_labels, 168)

# 3. Network architecture

## 3.1. ConvNet (Feature extraction)

Define the network architecture that will be used for training

- how many layers 
- which type of layer
    - Convolution: #channels, kernel size, activation
    - MaxPool: matrix size

Important: ConvNets take as input tensors of shape: (image_height, image_width, channels)

**Note:**

Convolutional layers learn local patterns (features that can appear anywhere in the image)

Dense layers learn global patterns.

In [0]:
network = models.Sequential()
network.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
network.add(layers.MaxPool2D((2, 2)))
network.add(layers.Conv2D(64, (3, 3), activation='relu'))
network.add(layers.MaxPool2D((2, 2)))
network.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [0]:
network.summary()

## 3.2. Classifier layer

In [0]:
network.add(layers.Flatten())
network.add(layers.Dense(64, activation='relu'))
network.add(layers.Dense(10, activation='softmax'))

In [0]:
network.summary()

# 4. Compilation Step

In the compilation step we define the:

- the loss function
- the optimizer
- the evaluation metric

In [0]:
network.compile(optimizer='rmsprop',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

# 5. Data Preparation

Before feeding the data into the network for training, we make sure it is formatted properly.

## Prepare the images

In [0]:
train_images_reshaped = train_images.reshape((60000, 28, 28, 1))
test_images_reshaped = test_images.reshape((10000, 28, 28, 1))

In [0]:
train_images_reshaped.shape

In [0]:
train_images_transformed = train_images_reshaped.astype('float32') / 255
test_images_transformed = test_images_reshaped.astype('float32') / 255

## Prepare the labels

In [0]:
train_labels_categorical = to_categorical(train_labels)
test_labels_categorical = to_categorical(test_labels)

In [0]:
train_labels_categorical[0]

# 6. Network summary

In [0]:
network.summary()

# 7. Train the network

Feed the training images and labels to the network.

Two additional parameters need to be supplied:

- epochs: how many times the network will look at the entire dataset. 
- batch_size: how many images will be put through the network at one time.

In [0]:
network.fit(train_images_transformed, train_labels_categorical, epochs=5, batch_size=128)

# 8. Test the network

Use the test set (which the network has not seen yet) to test how well the network will perform on images it has not seen yet:

In [0]:
test_loss, test_acc = network.evaluate(test_images_transformed, test_labels_categorical)

In [0]:
print('test_acc; ', test_acc)